In [1]:
#In this file, we used KNN to train the model and obtain the corresponding accuracy.
#First we adjust the parameters to get the best number of neighbor in KNN according to 
#the accuracy.
#Then, we eliminate the parent-children relationship and used the KNN to train again
#Lastly, we used PCA to lower the dimension and train again.
#After the step above, we analyzed which feature-Engineering work can improve the accuracy 
#of the model and we can find out whether KNN is suitable for the experiment.

In [4]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
#import xgboost as xgb
#import lightgbm as lgb
#from sklearn.metrics import r2_score
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsClassifier 

import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
from scipy.stats import norm 
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings

In [7]:
datapath='/Users/manshanlin/Desktop/dataset/'
X1=pd.read_csv(datapath+'word_vector_df.csv')
y1=pd.read_csv(datapath+'y.csv')
y1=y1['cuisine']

FileNotFoundError: File b'/Users/manshanlin/Desktop/dataset/word_vector_df.csv' does not exist

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X1, y1, test_size=0.2, random_state=42)

NameError: name 'X1' is not defined

In [14]:
knn=KNeighborsClassifier(n_neighbors=900)
knn.fit(X_train.values,y_train.values)
predictions=knn.predict(X_test.values)

In [15]:

sum(predictions==y_test)/len(y_test)


0.19617552948346792

In [11]:

#Try different parameters of how many neighbors
max=0
index=0
for i in range(1,10):

	knn=KNeighborsClassifier(n_neighbors=i)
	knn.fit(X_train,y_train)
	prediction=knn.predict(X_test)
	accuracy=(sum(prediction==y_test)/len(y_test))
	if accuracy>max:
		index=i
		max=accuracy
print(index)
print(max)
print('The best number of neignbor among 1-10 is ',index)

9
0.13187566661587688


In [12]:
max=0
index=0
for i in range(1,10):

	knn=KNeighborsClassifier(n_neighbors=i*10)
	knn.fit(X_train,y_train)
	prediction=knn.predict(X_test)
	accuracy=(sum(prediction==y_test)/len(y_test))
	if accuracy>max:
		index=i*10
		max=accuracy
print(index)
print(max)
print('The best number of neighbor among 10-100 is ',index )
print('The accuracy is ', max)

90
0.18451927472192595


In [13]:
#Eliminate the parent-son relationship 

def getChildren(dataF):
	column=list(dataF.columns)
	Children=[]
	for i in column:
		if ' ' not in i:
			Children.append(i)
	return Children

def UpdateVector1(dataF):
	children=getChildren(dataF)
	column=list(dataF.columns)
	deletion=[]
	for i in children:
		for j in column:
			if i!=j and i in j:
				dataF[i]=dataF[i]|dataF[j]
				deletion.append(j)
	for i in deletion:
		if i in dataF.columns:
			del dataF[i]
	return dataF

X1=UpdateVector1(X1)
knn=KNeighborsClassifier(n_neighbors=90)
knn.fit(X_train.values,y_train.values)
predictions=knn.predict(X_test.values)


print('The accuracy after eliminating parent-children relationship is',)
print(sum(predictions==y_test)/len(y_test))







0.18451927472192595


In [16]:
from sklearn.decomposition import PCA


In [18]:

pca1 = PCA()
pca1.fit(X1)
X1 = pca1.transform(X1)
X_train, X_test, y_train, y_test = train_test_split(
                                    X1, y1, random_state=42, test_size=.33)

In [20]:
knn=KNeighborsClassifier(n_neighbors=90)
knn.fit(X_train,y_train)
predictions=knn.predict(X_test)


print('The accuracy after eliminating parent-children relationship is',)
print(sum(predictions==y_test)/len(y_test))


The accuracy after eliminating parent-children relationship is
0.18512875209507848


In [ ]:
#In conclusion, we find out that no matter how we adjust the parameters and using
#different techniques to deal with the data, the model performance with KNN is not 
#satisfied(under 20%). So in this specific problem, we may have to try other techniques such as logistic
#regression, SVM and so on...